In [31]:
import bilsm_crf_model

EPOCHS = 10
model, (train_x, train_y), (test_x, test_y) = bilsm_crf_model.create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         851600    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
crf_2 (CRF)                  (None, None, 7)           1470      
Total params: 1,093,870
Trainable params: 1,093,870
Non-trainable params: 0
_________________________________________________________________


D:\Anaconda\envs\tfenv\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
D:\Anaconda\envs\tfenv\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [32]:
# train model
model.fit(train_x, train_y,batch_size=16,epochs=EPOCHS, validation_data=[test_x, test_y])
model.save('./crf.h5')

Train on 50658 samples, validate on 4631 samples
Epoch 1/10
50658/50658 [==============================] - 764s 15ms/step - loss: 4.0580 - crf_viterbi_accuracy: 0.9526 - val_loss: 7.8845 - val_crf_viterbi_accuracy: 0.9698
Epoch 2/10
50658/50658 [==============================] - 747s 15ms/step - loss: 3.9707 - crf_viterbi_accuracy: 0.9781 - val_loss: 7.8648 - val_crf_viterbi_accuracy: 0.9729
Epoch 3/10
50658/50658 [==============================] - 779s 15ms/step - loss: 3.9577 - crf_viterbi_accuracy: 0.9841 - val_loss: 7.8596 - val_crf_viterbi_accuracy: 0.9761
Epoch 4/10
50658/50658 [==============================] - 752s 15ms/step - loss: 3.9513 - crf_viterbi_accuracy: 0.9881 - val_loss: 7.8595 - val_crf_viterbi_accuracy: 0.9780
Epoch 5/10
50658/50658 [==============================] - 771s 15ms/step - loss: 3.9473 - crf_viterbi_accuracy: 0.9910 - val_loss: 7.8610 - val_crf_viterbi_accuracy: 0.9794
Epoch 6/10
50658/50658 [==============================] - 722s 14ms/step - loss: 3.944

In [33]:
import bilsm_crf_model
import process_data
import numpy as np

model, (vocab, chunk_tags) = bilsm_crf_model.create_model(train=False)
predict_text = '中华人民共和国国务院总理周恩来在外交部长陈毅的陪同下，连续访问了埃塞俄比亚等非洲10国以及阿尔巴尼亚'
str, length = process_data.process_data(predict_text, vocab)
model.load_weights('./crf.h5')
raw = model.predict(str)[0][-length:]
result = [np.argmax(row) for row in raw]
result_tags = [chunk_tags[i] for i in result]

per, loc, org = '', '', ''

for s, t in zip(predict_text, result_tags):
    if t in ('B-PER', 'I-PER'):
        per += ' ' + s if (t == 'B-PER') else s
    if t in ('B-ORG', 'I-ORG'):
        org += ' ' + s if (t == 'B-ORG') else s
    if t in ('B-LOC', 'I-LOC'):
        loc += ' ' + s if (t == 'B-LOC') else s

print(['person:' + per, 'location:' + loc, 'organzation:' + org])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 200)         851600    
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
crf_3 (CRF)                  (None, None, 7)           1470      
Total params: 1,093,870
Trainable params: 1,093,870
Non-trainable params: 0
_________________________________________________________________
['person: 周恩来 陈毅', 'location: 埃塞俄比亚 非洲 阿尔巴尼亚', 'organzation: 中华人民共和国国务院 外交部']
